## Carrega bibliotecas necessárias e a base de dados processada

In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("../data/dataset_categorizado.csv")

## Encontra um modelo classificativo

In [58]:
# Prepara os dados para modelagem
y, x = [], []

# Define a regra para equalizar a frequência das classes
# Utiliza o ID do registro para coletar únicos
dist = {"1": 120, "0": 120}
ids = []
total = sum([dist[k] for k in dist.keys()])

while len(x) < total:
    i = np.random.randint(0, len(dataset))
    if dataset["ID"].values[i] in ids:
        continue
        
    classe = dataset["Response"].values[i]
    if dist[str(classe)] > 0:
        # Salva nas listas para modelagem
        y.append(classe)
        x.append([
            dataset["encoder_escolaridade"].values[i],
            dataset["encoder_estado_civil"].values[i],
            dataset["encoder_nascimento"].values[i],
            dataset["Kidhome"].values[i],
            dataset["Teenhome"].values[i],
            dataset["encoder_renda"].values[i]
        ])
        
        # Atualiza a distruibuição e a lista de ids
        dist[str(classe)] = dist[str(classe)] - 1
        ids.append(dataset["ID"].values[i])

# Separa o conjunto de treino e de teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=0)

# Executa a classificação
classificador = GaussianNB()
classificador.fit(x_treino, y_treino)

# Avalia acurácia
y_pred = classificador.predict(x_teste)
acuracia = accuracy_score(y_teste, y_pred)

# Para a matriz de confusão, o melhor é ter poucos erros com falso positivo
mc = confusion_matrix(y_teste, y_pred)
fp = mc[0][1] / len(y_teste)

print(f"Encontrada acurácia de {round(acuracia, 2)}")
print(f"Total de falsos positivos de {round(fp, 2)}")

Encontrada acurácia de 0.77
Total de falsos positivos de 0.06


### Persistindo o melhor teste

In [59]:
# Salva o modelo de classificação
file = open("../models/classificador.pickle", "wb")
pickle.dump(classificador, file)
file.close()

# Salva um txt com informações
file = open("../models/classificador.txt", "w+")
file.writelines([
    f"Modelo de classificação salvo em {datetime.now().isoformat()}.\n",
    f"Encontrada acurácia de {round(acuracia, 2)}\n",
    f"Total de falsos positivos de {round(fp, 2)}\n",
    "Campos utilizados:\n",
    "> encoder_escolaridade\n",
    "> encoder_estado_civil\n",
    "> encoder_nascimento\n",
    "> quantidade_criancas\n",
    "> quantidade_adolescentes\n",
    "> encoder_renda\n"
])
file.close()